In [2]:
from models.vqvae_model import VQVAEModel
# import libraries
import numpy as np
from termcolor import colored, cprint
# for display
from IPython.display import Image as ipy_image
from IPython.display import display

import torch
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
import torchvision.utils as vutils

from datasets.dataloader import CreateDataLoader, get_data_generator
from models.base_model import create_model
from utils.util_3d import render_sdf, render_mesh, sdf_to_mesh, save_mesh_as_gif
from utils.demo_util import SDFusionOpt

%load_ext autoreload
%autoreload 2
opt = SDFusionOpt(gpu_ids=0, seed=2023)
device = opt.device
ckpt_path = "vqvae_epoch-best.pth"
opt.init_model_args(ckpt_path=ckpt_path)
opt.model = "vqvae"
opt.dataset_mode = "snet"
vqvae = create_model(opt)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[*] SDFusionTestOption initialized.
Working with z of shape (1, 3, 16, 16, 16) = 12288 dimensions.
[*] weight successfully load from: vqvae_epoch-best.pth
[*] Model has been created: VQVAE-Model


In [3]:
vqvae.device

'cuda'

In [4]:
import h5py
from utils.util_3d import sdf_to_mesh
from utils.demo_util import get_partial_shape
vqvae.vqvae.to("cuda")
def inference(path, partial=False, return_z=False):

    h5_f = h5py.File(path, 'r')
    sdf = h5_f['pc_sdf_sample'][:].astype(np.float32)

    sdf = torch.Tensor(sdf).view(1, 64, 64, 64).unsqueeze(0)
    x_min, x_max = -1, 1
    y_min, y_max = 0, 1
    z_min, z_max = -1, 1
    xyz_dict = {'x': (x_min, x_max), 'y': (y_min, y_max), 'z': (z_min, z_max)}
    if partial:
        ret = get_partial_shape(sdf, xyz_dict)
        sdf, shape_missing = ret['shape_part'], ret['shape_missing']
    # print(sdf.shape)
    # sdf = sdf[:, :64, :64, :64]
    sdf = torch.clamp(sdf, min=-0.2, max=0.2)
    if return_z:
        
        z = vqvae.vqvae(sdf.cuda(), forward_no_quant=True, encode_only=True).detach()
        return z
        
    else:    
        vqvae.inference({'sdf': sdf})
        mesh_gen = sdf_to_mesh(vqvae.x_recon)
        mesh_orig = sdf_to_mesh(sdf)
        mesh_gen = trimesh.Trimesh(mesh_gen.verts_packed().cpu().numpy(), mesh_gen.faces_packed().cpu().numpy())
        mesh_orig = trimesh.Trimesh(mesh_orig.verts_packed().cpu().numpy(), mesh_orig.faces_packed().cpu().numpy())
        mesh_gen.fix_normals()
        mesh_orig.fix_normals()
    return mesh_gen, mesh_orig

In [7]:
import glob
from tqdm import tqdm
rng = np.random.default_rng(12345)
listo = glob.glob("../data/SDF_v1_64/*/*/*")
print(listo[0])
listo = np.array([l for l in listo if "ori" in l and ".npy" not in l and "motor" not in l])
for p in tqdm(listo):
    z = inference(p, return_z=True).cpu().numpy()
    np.save(p.replace("ori_sample.h5", "latent_code.npy"), z)

../data/SDF_v1_64/02958343/4271d0c8cdad6a84e572d3df5e4fc85f/ori_sample.h5


100%|█████████████████████████████████████| 38315/38315 [32:23<00:00, 19.71it/s]


In [6]:
listo = glob.glob("/home/amac/SDFusion/data/ShapeNet/SDF_v1_64/*/*/*")


In [ ]:
listo

In [ ]:
import trimesh
mesh_gen, mesh_orig = inference(listo[0], False)
mesh_gen.show()

In [ ]:
mesh_gen = sdf_to_mesh(pred)

In [ ]:
mesh_gen_orig = sdf_to_mesh(sdf.unsqueeze(0).cuda())

In [ ]:
sdf.device

In [ ]:
# initialize SDFusion model
ckpt_path = 'saved_ckpt/sdfusion-snet-all.pth'
opt.init_model_args(ckpt_path=ckpt_path)
SDFusion = create_model(opt)


In [ ]:
import trimesh
mesho = trimesh.Trimesh(mesh_gen.verts_packed().cpu().numpy(), mesh_gen.faces_packed().cpu().numpy())

In [ ]:
mesho.fix_normals()

In [ ]:
mesho.show()

In [ ]:
import trimesh
mesho = trimesh.Trimesh(mesh_gen_orig.verts_packed().cpu().numpy(), mesh_gen_orig.faces_packed().cpu().numpy())
mesho.fix_normals()
mesho.show()